In [1]:
def rc4(key):

  S = list(range(100))
  j = 0
  for i in range(100):
    j = (j + S[i] + key[i % len(key)]) % 100
    S[i], S[j] = S[j], S[i]

  i = j = 0
  while True:
    i = (i + 1) % 100
    j = (j + S[i]) % 100
    S[i], S[j] = S[j], S[i]
    yield S[(S[i] + S[j]) % 100]

def generate_rc4_sequence(seed, length):

  key = bytes(seed)
  cipher = rc4(key)
  return [next(cipher) for _ in range(length)]


In [2]:
def normalise(start_index, edge_size, random_sequence, candidates):

  seq_len = len(random_sequence)
  indices = []

  for i in range(edge_size):
    index = (start_index + i) % seq_len
    indices.append(random_sequence[index])

  # Normalize and round indices
  min_val = min(indices)
  max_val = max(indices)
  indices = [round((x - min_val) / (max_val - min_val) * (len(candidates) - 1)) for x in indices]
  unique_indices = set(indices)

  subset = list(set(candidates[i] for i in unique_indices))

  return subset


def generate_hypergraph(plaintext, edge_sequence, random_sequence):

    plaintext = list(plaintext)
    vertex_count = {char: 0 for char in plaintext}
    hypergraph = []
    hyperedge_vertices=[]

    for char in plaintext:
        hyperedge_vertices.append(char)
        vertex_count[char] += 1

    hyperedge_vertices.remove(plaintext[0])
    hypergraph.append(hyperedge_vertices)

    for i in range(1, len(plaintext)):

        edge_size = edge_sequence[i]
        hyperedge_vertices = []
        min_usage = min(vertex_count.values())
        candidates = [char for char, count in vertex_count.items() if count == min_usage]

        if (len(candidates)<edge_size):
            rem_char=edge_size-len(candidates)
            seq_code=random_sequence[rem_char-1]
            selection_index=sum(map(int,str(seq_code)))%len(plaintext)
            for char in plaintext[selection_index:selection_index+rem_char]:
              if char not in candidates:
                candidates.append(char)

        if len(candidates) > edge_size:
            tie_count = len(candidates)
            seq_code = random_sequence[tie_count-1] 
            selection_index = sum(map(int, str(seq_code))) % len(plaintext)
        
        candidates=normalise(selection_index,edge_size,random_sequence,candidates)

        for char in candidates:
            hyperedge_vertices.append(char)
            vertex_count[char]+= 1

        if plaintext[i] in hyperedge_vertices:
            hyperedge_vertices.remove(plaintext[i]) 
        hypergraph.append(list(hyperedge_vertices))

    return hypergraph

In [3]:
def hypergraph_encryption(plaintext, hypergraph):
    

  value_reference={}
  for char in plaintext:
    value_reference[char]=ord(char)

  ciphertext_list=[]
  c=0
  for hyperedge in hypergraph:
    xor_result=value_reference[plaintext[c]]
    for vertex in hyperedge:
      xor_result^=value_reference[vertex]
    ciphertext_list.append(xor_result)
    value_reference[plaintext[c]]= xor_result
    c+=1

  return ciphertext_list

In [6]:
import copy

def generate_hypergraph_indices(ciphertext, edge_sequence, random_sequence):

    ciphertext = list(ciphertext)
    vertex_count = {i: 0 for i in range(len(ciphertext))}
    hypergraph = []

    hyperedge=[]
    for index in range(len(ciphertext)):
        vertex_count[index] += 1
        hyperedge.append(index)
    hyperedge.remove(0)
    
    hypergraph.append(hyperedge)

    for i in range(1, len(ciphertext)):

        hyperedge=[]
        edge_size = edge_sequence[i]
        min_usage = min(vertex_count.values())
        candidates = [index for index, count in vertex_count.items() if count == min_usage]
        # print(candidates)

        if (len(candidates)<edge_size):
            rem_char=edge_size-len(candidates)
            seq_code=random_sequence[rem_char-1]
            selection_index=sum(map(int,str(seq_code)))%len(ciphertext)
            for index in range(selection_index,selection_index+rem_char):
              if index not in candidates:
                candidates.append(index)

        if len(candidates) > edge_size:
            tie_count = len(candidates)
            seq_code = random_sequence[tie_count-1] 
            selection_index = sum(map(int, str(seq_code))) % len(ciphertext)
        
        candidates=normalise(selection_index,edge_size,random_sequence,candidates)
        # print(candidates)

        for index in candidates:
            hyperedge.append(index)
            vertex_count[index]+= 1

        hyperedge_copy=copy.deepcopy(hyperedge)
        if i in hyperedge_copy:
            hyperedge_copy.remove(i)  
        hypergraph.append(hyperedge_copy)


    return reverse_XOR(hypergraph,ciphertext)

def reverse_XOR(hypergraph_indices,ciphertext):
    reference_value=dict(enumerate(ciphertext))
    print(reference_value)
    plaintext=[0]*len(ciphertext)
    for edge_index in range(len(ciphertext)-1,-1,-1):
        hyperedge=hypergraph_indices[edge_index]
        # print(hyperedge)
        plaintext_vertex=0
        for vertex in hyperedge:
            plaintext_vertex^=reference_value[vertex]
        plaintext_vertex^=ciphertext[edge_index]
        # print(plaintext_vertex)
        plaintext[edge_index]=plaintext_vertex
        reference_value[edge_index]=plaintext_vertex
        # print(reference_value)

    # print(reference_value)
    plaintext=[chr(char) for char in plaintext]
    return plaintext

# ciphertext=[17, 2, 122, 10, 23, 99]
# edge_sequence = [6, 3, 4, 2, 5, 2]
# random_sequence = [51, 99, 85, 26, 34, 55]
# hypergraph = generate_hypergraph_indices(ciphertext, edge_sequence, random_sequence)
# # quit()
# print(hypergraph)


In [7]:
def nr2(plaintext,edge_seq,seed):
    
    random_seq=generate_rc4_sequence(seed,len(plaintext))

    hypergraph=generate_hypergraph(plaintext,edge_seq,random_seq)

    ciphertext_list=hypergraph_encryption(plaintext,hypergraph)
    ciphertext=""
    for char in ciphertext_list:
        ciphertext+=str(char)+"x"

    decrypted_plaintext=generate_hypergraph_indices(ciphertext_list,edge_seq,random_seq)
    print("Encryption:")
    print()
    print("Plain Text: ", plaintext)
    print("Generated Hypergraph: ", hypergraph)
    print("Cipher Text: ",ciphertext)
    print()
    print("Decryption:")
    print()
    print("Encrypted Plain Text: ", decrypted_plaintext)


    return ciphertext
    


In [8]:
plaintext = input('Enter Plaintext: ')
edge_seq=input("Enter Key :").split(" ")
edge_seq = [int(num) for num in edge_seq]
seed=int(input('Enter Seed Value for random generator: '))
nr2(plaintext,edge_seq,seed)



Enter Plaintext:  xavier
Enter Key : 6 3 4 2 5 2
Enter Seed Value for random generator:  11


{0: 17, 1: 103, 2: 105, 3: 124, 4: 6, 5: 101}
Encryption:

Plain Text:  xavier
Generated Hypergraph:  [['a', 'v', 'i', 'e', 'r'], ['e', 'r', 'x'], ['a', 'i', 'x'], ['a', 'r'], ['a', 'i', 'x', 'v'], ['e', 'x']]
Cipher Text:  17x103x105x124x6x101x

Decryption:

Encrypted Plain Text:  ['x', 'a', 'v', 'i', 'e', 'r']


'17x103x105x124x6x101x'

In [11]:
import time
def test_speed():
    plaintext = input('Enter Plaintext: ')
    edge_seq=input("Enter Key :").split(" ")
    edge_seq = [int(num) for num in edge_seq]
    seed=int(input('Enter Seed Value for random generator: '))
    start_time = time.time()
    nr2(plaintext,edge_seq,seed)  
    end_time = time.time()
    combined_time = end_time - start_time
    print("Combined time:", combined_time)

In [12]:
if __name__ == "__main__":
    test_speed()

Enter Plaintext:  xavier
Enter Key : 6 3 4 2 5 2
Enter Seed Value for random generator:  11


{0: 17, 1: 103, 2: 105, 3: 124, 4: 6, 5: 101}
Encryption:

Plain Text:  xavier
Generated Hypergraph:  [['a', 'v', 'i', 'e', 'r'], ['e', 'r', 'x'], ['a', 'i', 'x'], ['a', 'r'], ['a', 'i', 'x', 'v'], ['e', 'x']]
Cipher Text:  17x103x105x124x6x101x

Decryption:

Encrypted Plain Text:  ['x', 'a', 'v', 'i', 'e', 'r']
Combined time: 0.0005724430084228516
